# Chapter 9: Metasurfaces

**PyMetaLearn - Metamaterial Physics Workbook**

---

## Learning Objectives

By the end of this chapter, you will:
- Understand the generalized laws of reflection and refraction
- Learn about phase gradient metasurfaces
- Derive the Pancharatnam-Berry phase
- Design a flat metalens
- Explore beam steering and holography applications

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from sympy import *
from IPython.display import display, Markdown, Math

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
init_printing(use_latex=True)

c = 3e8
print("Libraries loaded!")

---

## 9.1 What Are Metasurfaces?

**Metasurfaces** are 2D arrays of sub-wavelength scatterers ("meta-atoms") that manipulate light at an interface.

### Advantages over 3D Metamaterials:
- **Thin**: Single layer, easy to fabricate
- **Low loss**: Short propagation distance
- **Scalable**: Lithography-compatible
- **Versatile**: Control amplitude, phase, polarization

### Key Concept
Each meta-atom introduces a **local phase shift** φ(x, y) to the transmitted/reflected light.

By engineering φ(x, y), we can create:
- Flat lenses (metalenses)
- Beam steerers
- Holograms
- Polarization converters

In [ ]:
# Visualize meta-atom concept
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Meta-atom array
ax1.set_xlim(-2, 8)
ax1.set_ylim(-1, 3)

# Draw substrate
substrate = Rectangle((-2, -0.5), 10, 0.5, facecolor='lightgray', edgecolor='black')
ax1.add_patch(substrate)

# Draw meta-atoms (pillars of varying size → varying phase)
positions = [0, 1.5, 3, 4.5, 6]
widths = [0.2, 0.3, 0.4, 0.5, 0.6]
phases = ['0', 'π/4', 'π/2', '3π/4', 'π']
colors = plt.cm.hsv(np.linspace(0, 0.5, 5))

for pos, w, phase, color in zip(positions, widths, phases, colors):
    pillar = Rectangle((pos - w/2, 0), w, 1.2, facecolor=color, edgecolor='black', linewidth=1.5)
    ax1.add_patch(pillar)
    ax1.annotate(f'φ = {phase}', (pos, 1.5), ha='center', fontsize=10)

ax1.set_xlabel('Position x', fontsize=12)
ax1.set_title('Metasurface: Array of Meta-atoms with Varying Phase', fontsize=14)
ax1.set_aspect('equal')
ax1.axis('off')

# Right: Phase profile for a lens
x = np.linspace(-5, 5, 500)
f = 10  # Focal length in wavelengths
k0 = 1

# Lens phase profile
phi_lens = -k0 * (np.sqrt(x**2 + f**2) - f)
phi_lens_wrapped = np.mod(phi_lens + np.pi, 2*np.pi) - np.pi

ax2.plot(x, phi_lens, 'b-', linewidth=2, label='Unwrapped phase')
ax2.plot(x, phi_lens_wrapped, 'r--', linewidth=2, label='Wrapped (0 to 2π)')
ax2.set_xlabel('Position x (λ)', fontsize=12)
ax2.set_ylabel('Phase φ(x) (rad)', fontsize=12)
ax2.set_title('Metalens Phase Profile (f = 10λ)', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 9.2 Generalized Snell's Law

Yu & Capasso (2011) derived the **generalized laws of reflection and refraction** when a phase gradient exists at an interface.

### Classical Snell's Law
$$n_1 \sin\theta_i = n_2 \sin\theta_t$$

### Generalized Snell's Law
$$\boxed{n_2 \sin\theta_t - n_1 \sin\theta_i = \frac{\lambda_0}{2\pi} \frac{d\phi}{dx}}$$

The extra term $\frac{d\phi}{dx}$ is the **phase gradient** imparted by the metasurface!

In [ ]:
# Symbolic derivation
theta_i, theta_t, theta_r = symbols('theta_i theta_t theta_r', real=True)
n1, n2, lambda_0, phi = symbols('n_1 n_2 lambda_0 phi', positive=True)
x = symbols('x', real=True)

display(Markdown("### Derivation from Fermat's Principle"))
display(Math(r"\text{Optical path length: } L = n_1 \sin\theta_i \cdot x + \phi(x) \cdot \frac{\lambda_0}{2\pi}"))

display(Markdown("### For stationary phase (Fermat's principle): dL/dx = 0"))
display(Math(r"n_1 \sin\theta_i \cdot dx + \frac{\lambda_0}{2\pi} d\phi = n_2 \sin\theta_t \cdot dx"))

display(Markdown("### Generalized Snell's Law:"))
display(Math(r"\boxed{n_2 \sin\theta_t - n_1 \sin\theta_i = \frac{\lambda_0}{2\pi} \frac{d\phi}{dx}}"))

In [ ]:
# Visualize generalized refraction
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Without phase gradient (classical)
ax1.axhline(y=0, color='black', linewidth=2)
ax1.annotate('n₁ = 1 (air)', (-2, 1.5), fontsize=12)
ax1.annotate('n₂ = 1.5 (glass)', (-2, -1.5), fontsize=12)

theta_i_deg = 30
theta_i_rad = np.radians(theta_i_deg)
n1_val, n2_val = 1.0, 1.5
theta_t_classical = np.arcsin(n1_val * np.sin(theta_i_rad) / n2_val)

# Incident ray
ax1.arrow(-2*np.sin(theta_i_rad), 2*np.cos(theta_i_rad), 
          2*np.sin(theta_i_rad), -2*np.cos(theta_i_rad),
          head_width=0.15, head_length=0.1, fc='blue', ec='blue')
# Transmitted ray
ax1.arrow(0, 0, 2*np.sin(theta_t_classical), -2*np.cos(theta_t_classical),
          head_width=0.15, head_length=0.1, fc='red', ec='red')
# Normal
ax1.axvline(x=0, color='gray', linestyle='--', linewidth=1)

ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)
ax1.set_title(f'Classical Snell\'s Law\nθᵢ = {theta_i_deg}°, θₜ = {np.degrees(theta_t_classical):.1f}°', fontsize=14)
ax1.set_aspect('equal')
ax1.axis('off')

# With phase gradient (anomalous refraction)
ax2.axhline(y=0, color='black', linewidth=2)

# Add phase gradient visualization
gradient_x = np.linspace(-2.5, 2.5, 20)
for gx in gradient_x:
    color = plt.cm.hsv(0.5 + gx/5)  # Color varies with position
    ax2.plot([gx, gx], [-0.1, 0.1], color=color, linewidth=3)

ax2.annotate('Phase gradient dφ/dx', (0, 0.4), ha='center', fontsize=11, 
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# With anomalous refraction (phase gradient = 0.5 k0)
dphi_dx = 0.5 * (2*np.pi / 500e-9)  # Phase gradient
lambda_0_val = 500e-9
sin_theta_t_anom = n1_val * np.sin(theta_i_rad) / n2_val + lambda_0_val / (2*np.pi) * dphi_dx / n2_val
sin_theta_t_anom = np.clip(sin_theta_t_anom, -1, 1)  # Ensure valid
theta_t_anom = np.arcsin(sin_theta_t_anom)

# Incident ray
ax2.arrow(-2*np.sin(theta_i_rad), 2*np.cos(theta_i_rad), 
          2*np.sin(theta_i_rad), -2*np.cos(theta_i_rad),
          head_width=0.15, head_length=0.1, fc='blue', ec='blue')
# Anomalous transmitted ray
ax2.arrow(0, 0, 2*np.sin(theta_t_anom), -2*np.cos(theta_t_anom),
          head_width=0.15, head_length=0.1, fc='green', ec='green')
# Classical (for comparison, dashed)
ax2.arrow(0, 0, 1.5*np.sin(theta_t_classical), -1.5*np.cos(theta_t_classical),
          head_width=0.1, head_length=0.08, fc='red', ec='red', alpha=0.3, linestyle='--')

ax2.set_xlim(-3, 3)
ax2.set_ylim(-3, 3)
ax2.set_title(f'Generalized Snell\'s Law\nAnomalous refraction with dφ/dx > 0', fontsize=14)
ax2.set_aspect('equal')
ax2.axis('off')

ax2.annotate('Classical (dashed)', (1.2, -1.5), fontsize=10, color='red')
ax2.annotate('Anomalous (solid)', (2, -0.5), fontsize=10, color='green')

plt.tight_layout()
plt.show()

---

## 9.3 Designing a Metalens

A **metalens** (flat lens) focuses light by imparting a **hyperbolic phase profile**:

$$\phi(x, y) = -\frac{2\pi}{\lambda}\left(\sqrt{x^2 + y^2 + f^2} - f\right)$$

Where:
- $f$ = focal length
- $(x, y)$ = position on the metasurface

### Physical Interpretation
This phase compensates for the different path lengths from each point on the surface to the focus.

In [ ]:
# Design a metalens
lambda_0 = 600e-9  # 600 nm
f = 100e-6  # 100 μm focal length
D = 50e-6  # 50 μm diameter

# Create grid
x = np.linspace(-D/2, D/2, 500)
y = np.linspace(-D/2, D/2, 500)
X, Y = np.meshgrid(x, y)
R = np.sqrt(X**2 + Y**2)

# Phase profile
k0 = 2 * np.pi / lambda_0
phi = -k0 * (np.sqrt(R**2 + f**2) - f)

# Wrap to [0, 2π]
phi_wrapped = np.mod(phi, 2*np.pi)

# Circular aperture
mask = R <= D/2
phi_wrapped[~mask] = np.nan

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Phase profile
im1 = axes[0].imshow(phi_wrapped, extent=[-D/2*1e6, D/2*1e6, -D/2*1e6, D/2*1e6], 
                     cmap='hsv', origin='lower')
axes[0].set_xlabel('x (μm)', fontsize=12)
axes[0].set_ylabel('y (μm)', fontsize=12)
axes[0].set_title(f'Metalens Phase Profile\n(f = {f*1e6:.0f} μm, D = {D*1e6:.0f} μm)', fontsize=12)
plt.colorbar(im1, ax=axes[0], label='Phase (rad)')

# Cross-section
center_idx = len(y) // 2
axes[1].plot(x*1e6, phi_wrapped[center_idx, :], 'b-', linewidth=2)
axes[1].set_xlabel('Position x (μm)', fontsize=12)
axes[1].set_ylabel('Phase (rad)', fontsize=12)
axes[1].set_title('Phase Cross-Section (y = 0)', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Numerical aperture and spot size
NA = np.sin(np.arctan(D/(2*f)))
spot_airy = 0.61 * lambda_0 / NA * 1e9  # Airy disk radius in nm

info_text = f"""Metalens Parameters:
━━━━━━━━━━━━━━━━━━━━━
Wavelength: {lambda_0*1e9:.0f} nm
Focal length: {f*1e6:.0f} μm
Diameter: {D*1e6:.0f} μm
Numerical Aperture: {NA:.2f}
Diffraction-limited spot: {spot_airy:.0f} nm
━━━━━━━━━━━━━━━━━━━━━
"""

axes[2].text(0.1, 0.5, info_text, fontsize=12, family='monospace',
            verticalalignment='center', transform=axes[2].transAxes,
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
axes[2].axis('off')
axes[2].set_title('Design Summary', fontsize=12)

plt.tight_layout()
plt.show()

---

## 9.4 Pancharatnam-Berry Phase

A powerful technique for metasurface design uses the **geometric phase** (Pancharatnam-Berry phase).

### Key Idea
When circularly polarized light passes through a rotated half-wave plate:

$$\phi_{PB} = \pm 2\theta$$

Where:
- $\theta$ = rotation angle of the meta-atom
- $+$ for RCP → LCP conversion
- $-$ for LCP → RCP conversion

### Advantages
- **Broadband**: Phase depends only on geometry, not wavelength!
- **Robust**: Manufacturing tolerances don't affect phase
- **Simple design**: Just rotate identical structures

In [ ]:
# Visualize PB phase metasurface
fig, ax = plt.subplots(figsize=(12, 4))

# Create array of rotated nanofins
N = 20  # Number of elements
positions = np.linspace(0, 10, N)

# Linear phase gradient = beam steering
steering_angle = 30  # degrees
target_phase = np.linspace(0, 2*np.pi, N)  # Full 2π across aperture
rotation_angles = target_phase / 2  # PB phase = 2θ

for pos, theta in zip(positions, rotation_angles):
    # Draw rotated rectangle (nanofin)
    w, h = 0.15, 0.4
    rect = plt.Rectangle((pos - w/2, -h/2), w, h, 
                         angle=np.degrees(theta), 
                         facecolor=plt.cm.hsv(theta/np.pi),
                         edgecolor='black', linewidth=0.5,
                         rotation_point='center')
    ax.add_patch(rect)

ax.set_xlim(-0.5, 10.5)
ax.set_ylim(-1.5, 1.5)
ax.set_xlabel('Position', fontsize=12)
ax.set_title('Pancharatnam-Berry Metasurface: Rotated Nanofins for Beam Steering', fontsize=14)
ax.set_aspect('equal')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.3)

# Color legend
sm = plt.cm.ScalarMappable(cmap='hsv', norm=plt.Normalize(0, 180))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.02)
cbar.set_label('Rotation angle θ (degrees)', fontsize=11)

plt.tight_layout()
plt.show()

print("💡 Phase = 2θ: Rotating nanofin by 90° → 180° (π) phase shift")
print("   This creates a linear phase gradient → beam steering!")

---

## 9.5 Applications of Metasurfaces

| Application | Phase Profile | Example |
|-------------|--------------|----------|
| **Beam steering** | Linear: φ(x) = kₓx | LiDAR, optical switches |
| **Focusing (metalens)** | Hyperbolic | Cameras, microscopy |
| **Holography** | Computed (CGH) | Displays, security |
| **Vortex beam** | Azimuthal: φ = lθ | Optical tweezers, comms |
| **Polarization control** | Spatially varying | Polarimetry, display |

In [ ]:
# Gallery of metasurface phase profiles
fig, axes = plt.subplots(2, 3, figsize=(14, 9))

x = np.linspace(-1, 1, 300)
y = np.linspace(-1, 1, 300)
X, Y = np.meshgrid(x, y)
R = np.sqrt(X**2 + Y**2)
Theta = np.arctan2(Y, X)

profiles = [
    ('Lens (f=2)', -np.pi * (np.sqrt(R**2 + 4) - 2)),
    ('Beam Steering (30°)', 2*np.pi * 0.5 * X),
    ('Axicon (ring focus)', -np.pi * R),
    ('Vortex (l=1)', Theta),
    ('Vortex (l=3)', 3 * Theta),
    ('Cubic (Airy beam)', (X**3 + Y**3) * 3),
]

for ax, (title, phi) in zip(axes.flat, profiles):
    phi_wrapped = np.mod(phi, 2*np.pi)
    im = ax.imshow(phi_wrapped, extent=[-1, 1, -1, 1], cmap='hsv', origin='lower')
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.suptitle('Metasurface Phase Profiles for Different Applications', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## 9.6 State of the Art

### Recent Breakthroughs (2020-2024)

1. **Commercial metalenses**: Metalenz Inc. shipping for smartphones
2. **Achromatic metalenses**: Dispersion-corrected for broadband imaging
3. **Tunable metasurfaces**: Phase-change materials, MEMS, liquid crystals
4. **Nonlinear metasurfaces**: Frequency conversion, harmonic generation
5. **Quantum metasurfaces**: Single-photon manipulation, entanglement

---

## 9.7 Summary & Key Takeaways

1. **Metasurfaces** are 2D arrays of sub-λ scatterers
2. **Generalized Snell's law** includes a phase gradient term
3. **Metalenses** use hyperbolic phase for focusing
4. **Pancharatnam-Berry phase** enables broadband control via rotation
5. **Applications**: Lensing, steering, holography, polarization

---

## 📝 Exercises

1. **Design** a metalens with f = 1 mm and D = 0.5 mm for λ = 1.55 μm
2. **Calculate** the steering angle for a metasurface with period Λ = λ/2
3. **Plot** the phase profile for a hologram that creates the letter 'A'
4. **Explain** why PB phase metasurfaces only work for circularly polarized light

---

## 📚 References

1. N. Yu et al., "Light Propagation with Phase Discontinuities", *Science* **334**, 333 (2011)
2. M. Khorasaninejad et al., "Metalenses at visible wavelengths", *Science* **352**, 1190 (2016)
3. W.T. Chen et al., "A broadband achromatic metalens", *Nature Nanotechnology* **13**, 220 (2018)

---

**← [08 - Optical Metamaterials](08_optical_metamaterials.ipynb) | [10 - Applications](10_applications.ipynb) →**